<a href="https://colab.research.google.com/github/Mbigdeli2003/Fault_detection_training/blob/main/Fault_Detection_HW3_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Load and split data
data = pd.read_csv('TEP_data.csv')  # Replace with TEP dataset
X, y = data.drop('fault', axis=1), data['fault']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Models for SVM and KNN
models = {
    'SVM': SVC(kernel='rbf', C=1.0),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

# Train, evaluate, and compare SVM and KNN
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred, average='weighted'),
        'Confusion Matrix': confusion_matrix(y_test, y_pred)
    }

# Semi-supervised SVM
semi_supervised_model = SelfTrainingClassifier(SVC(kernel='rbf', C=1.0), max_iter=10)
semi_supervised_model.fit(X_train, y_train)
y_pred_semi = semi_supervised_model.predict(X_test)

# Random Forest and Ensemble Models
ensemble_models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Bagging': BaggingClassifier(base_estimator=SVC(kernel='linear'), n_estimators=10, random_state=42),
    'Boosting': GradientBoostingClassifier(n_estimators=50, learning_rate=0.1, random_state=42)
}
for name, model in ensemble_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {'Accuracy': accuracy_score(y_test, y_pred)}

# Plot confusion matrices
for name, metrics in results.items():
    sns.heatmap(metrics['Confusion Matrix'], annot=True, fmt='d', cmap='Blues')
    plt.title(f'{name} Confusion Matrix')
    plt.show()

# Compare model accuracy
plt.bar(results.keys(), [metrics['Accuracy'] for metrics in results.values()])
plt.ylabel('Accuracy')
plt.title('Model Accuracy Comparison')
plt.show()
